## Homework 8

Part 1 is for discussions only.

Part 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

In [4]:
ltcm = pd.read_excel('ltcm_exhibits_data.xlsx', sheet_name=1,skiprows=2).rename(columns={"Unnamed: 0": "Date"})\
                                                                             .set_index('Date').dropna()
ltcm.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01 00:00:00,1.1,-0.011,-0.013,0.99
1994-04-01 00:00:00,1.1,0.014,0.008,1.00
1994-05-01 00:00:00,1.2,0.068,0.053,1.05
1994-06-01 00:00:00,1.2,-0.039,-0.029,1.02
1994-07-01 00:00:00,1.4,0.116,0.084,1.10


In [9]:
SPY = pd.read_excel('gmo_analysis_data.xlsx', sheet_name='returns (total)').set_index('Date')
rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name='risk-free rate').set_index('Date')

SPY['rf'] = rf['US3M']
SPY_ex = SPY.subtract(SPY['rf'], axis=0)['SPY'].to_frame('SPY ex')

SPY_ex.head()

,SPY ex
Date,
1993-02-28,0.0082
1993-03-31,0.0199
1993-04-30,-0.0281
1993-05-31,0.0244
1993-06-30,0.0011


In [18]:
ltcm.index = SPY_ex.loc['1994-03':'1998-07'].index
ltcm_ex = ltcm.subtract(SPY['rf'], axis = 0).dropna().drop(columns = ['Index of Net Performance'])
ltcm_ex['SPY (excess)'] = SPY_ex['SPY ex']
ltcm['rf']=SPY['rf']

ltcm.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance,rf
Date,,,,,
1994-03-31,1.1,-0.011,-0.013,0.99,0.0030
1994-04-30,1.1,0.014,0.008,1.00,0.0033
1994-05-31,1.2,0.068,0.053,1.05,0.0036
1994-06-30,1.2,-0.039,-0.029,1.02,0.0036
1994-07-31,1.4,0.116,0.084,1.10,0.0037


In [19]:
ltcm_ex.columns = ['Fund Capital ($billions)', 'Gross Performance (excess)', 'Net Performance (excess)', 'SPY (excess)']

ltcm_ex.head()

,Fund Capital ($billions),Gross Performance (excess),Net Performance (excess),SPY (excess)
Date,,,,
1994-03-31,1.0970,-0.0140,-0.0160,-0.0449
1994-04-30,1.0967,0.0107,0.0047,0.0079
1994-05-31,1.1964,0.0644,0.0494,0.0123
1994-06-30,1.1965,-0.0425,-0.0326,-0.0264
1994-07-31,1.3963,0.1123,0.0803,0.0287


### 2. LTCM Risk Decomposition

#### 2.1

In [20]:
def summary_stats(df, annual_fac=12):
    stat_df = (df.mean()*annual_fac).to_frame('Mean')
    stat_df['Vol'] = df.std()*np.sqrt(annual_fac)
    stat_df['Sharpe'] = stat_df['Mean']/stat_df['Vol']
    
    return stat_df

def tail_risk_stats(df, annual_fac=12):
    tr_df = df.skew().to_frame('Skew')
    tr_df['Kurtosis'] = df.kurtosis()
    tr_df['VaR (0.05)'] = df.quantile(0.05)
    return tr_df

2.1 (a)

In [21]:
summary_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])

,Mean,Vol,Sharpe
Gross Performance (excess),0.2421,0.1362,1.7769
Net Performance (excess),0.1554,0.1118,1.3901
SPY (excess),0.1738,0.1123,1.5479


2.1 (b)

In [22]:
tail_risk_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])

,Skew,Kurtosis,VaR (0.05)
Gross Performance (excess),-0.2877,1.5866,-0.0304
Net Performance (excess),-0.8102,2.9269,-0.0264
SPY (excess),-0.4335,-0.3620,-0.0464


2.1 (c)

The above summary stats relect well for the exhibits. Post-1997 the stats looks even better. Sharpe is relatively very high. Volatility is similar to SPY, and minimum return is satisfactory. SPY fared well during the period, so LTCM's outperformance is not an outlier.

The tail-risk statistics look worse for LTCM than for SPY post 1998.

For August crash, it underperformed SPY, esp the tail-risk with no significant difference between Gross and Net. Despite lagging Gross, he net returns still deliver a excellent Sharpe and overall stats. Overall they lagged SPY.

#### 2.2 

2.2 (a)

In [24]:
def regress(y, X, intercept = True, annual_fac=12):
    if intercept == True:
        X_ = sm.add_constant(X)
        reg = sm.OLS(y, X_).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc[r'$R^{2}$'] = reg.rsquared
        reg_df.loc['const'] *= annual_fac
    else:
        reg = sm.OLS(y, X).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc[r'$R^{2}$'] = reg.rsquared
    
    return reg_df

In [25]:
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex['SPY (excess)']) 

,Regression Parameters
const,0.1315
SPY (excess),0.1371
$R^{2}$,0.0190


2.2 (b) No. Correlation to the market is small with small beta.

2.2 (c) Yes, LTCM delivers significant excess returns above expectations from market exposure. The alpha of the regression is large (13.15% annualized).

#### 2.3

2.3 (a)

In [27]:
ltcm_ex['SPY (excess) squared'] = ltcm_ex['SPY (excess)'] **2

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','SPY (excess) squared']]) 

,Regression Parameters
const,0.1550
SPY (excess),0.1669
SPY (excess) squared,-1.9267
$R^{2}$,0.0243


2.3 (b) The quadratic market factor does not add to R-squared much. Quadratic movements of SPY are not significant in explaining LTCM variation.

2.3 (c) LTCM assesses to be short market options due to its negative exposure to market volatility (this is what the quadratic market factor represents).

2.3 (d) It is negative to the degree it has quadratic exposure. LTCM underperforms particularly large SPY movements in any direction. So LTCM has negative exposure to market volatility.

#### 2.4 

2.4 (a)

In [29]:
k1 = .03
k2 = -.03

ltcm_ex['Up'] = (ltcm_ex['SPY (excess)'] - k1).clip(0)
ltcm_ex['Down'] = (k2 - ltcm_ex['SPY (excess)']).clip(0)

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','Up','Down']])

,Regression Parameters
const,0.1012
SPY (excess),0.4666
Up,-0.7821
Down,1.2896
$R^{2}$,0.0555


2.4 (b)  Beta(up) is negative so LTCM appears short the call-like factor. Beta(down) is positive so LTCM appears long the put-like factor.

2.4 (c) The magnitude of the Beta(down) is larger, meaning that the put-like factor moves LTCM more.

2.4 (d) The previous problem concluded that LTCM is negatively exposed to market volatility. This volatility exposure must come more from being short the market's upside, as LTCM is short the call-like factor and long the put-like factor.

### 3. FX Carry trade

In [34]:
rf = pd.read_excel('fx_carry_data.xlsx', sheet_name = 1).set_index('DATE')

log_rf = np.log(1 + rf)

log_rf.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3458e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3226e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4270e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8953e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.4997e-05


In [36]:
fx = pd.read_excel('fx_carry_data.xlsx', sheet_name = 2).set_index('DATE')

log_fx = np.log(fx)

log_fx.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


In [37]:
log_rf_ex = log_rf.subtract(log_rf['USD1M'], axis=0)[log_rf.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()

log_rf_ex_col.columns = log_fx.columns

ret = (log_rf_ex_col.shift() + log_fx.diff()).dropna()

ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160
1999-06-30,-0.0157,-0.0128,-0.0211,-0.0045


#### 3.1 (a) (b) (c)

In [38]:
summary_stats(ret)

,Mean,Vol,Sharpe
USUK,-0.0035,0.0863,-0.0406
USEU,-0.0044,0.0947,-0.0459
USSZ,0.0043,0.0988,0.0437
USJP,-0.0174,0.0915,-0.1903


#### 3.2

3.2 (a) 

The results above are evidence against UIP. UIP implies that the mean excess return should be zero. The USD and JPY trade seems to have a mean excess return significantly different from zero.

The evidence from the other currencies is not clear. Their mean excess return is not exactly zero, but they are so close to zero that they may not be statistically significant.

UIP makes no implication about the volatility of these excess returns.

3.2 (b) 
Being long CHF would've delivered a small positive mean excess log return, but a poor Sharpe.
The best trade would have been to be short JPY and long USD. This would deliver a small but positive mean return and Sharpe.

3.2 (c) A long position in three of the currencies had negative mean excess (log) returns.

#### 3.3

3.3 (a)

In [39]:
ys = log_fx.diff().dropna()
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = [r'$\alpha^{i}$',r'$\beta^{i}$',r'$R^{2}$'])

for i in range(0,len(ys.columns)):
    fx_reg_pred[ys.columns[i]] = regress(ys[ys.columns[i]],Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
$\alpha^{i}$,-0.0059,0.0070,0.0436,-0.0060
$\beta^{i}$,0.4858,-1.2564,-1.6466,0.3715
$R^{2}$,0.0004,0.0026,0.0039,0.0005


3.3 (b) i. Expect a stronger EU and SZ due to negative betas.

3.3 (b) ii. Expect weaker UK and JP, due to their positive betas.

3.3 (b) iii. Strongest for USUK and USJP, due to alphas closest to zero

#### 3.4

In [40]:
alphas = (fx_reg_pred.loc[r'$\alpha^{i}$'] / 12)
betas = fx_reg_pred.loc[r'$\beta^{i}$']
Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas - 1) * Xs)

ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030


3.4 (a)

In [41]:
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0])/len(ret_ex_forecast)) * 100

ret_ex_forecast

month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0


3.4 (b) 

The static carry-trade in 3.3 found that UK and EU have negative mean excess (log) returns. However, as per the dynamic carry trade the time-varying forecast is positive.

The forecast is negative throughout the sample for JP due to the much higher interest rate in USD which is not offset by appreciation of the Yen to the USD.

3.4 (c) 
Just as in HW7, we can use forecasts of excess returns to construct trading positions, (weights,) which vary with the forecasts. The resulting trading strategy should be better than the static carry trade.

That is, we could time the magnitude and direction of the currency trades instead of being passively 100% invested in a currency according to the evidence in 3.3.

It is analagous to timing the trading of SPY according to the Earnings-Price Ratio instead of being passively 100% invested in SPY.

Of course, it is an open question as to whether this will work given the weak R-squared in-sample, let alone out-of-sample.

_Sources: Course github, homework problems and 2021 solution guides._